In [ ]:
import bmll2 as b2
from bmll2 import reference, Security, NormalisedSecurity, SparkHelper, get_market_data, get_market_data_range, VenueMarketError, get_market_tables, save_spark_dataframe, load_spark_dataframe

In [ ]:
import auxiliary_functions as af

import random
import math
import pandas as pd
import numpy as np
from pandas import StringDtype
import gc

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.ticker import LogFormatterSciNotation
from scipy.stats import linregress
from scipy.optimize import curve_fit
from scipy.stats import t

## GRT

The impact decay of metaorders from GRT is plotted below along with the fitted curve. The point estimates of the fitted values are tabulated

In [ ]:
ticker              = 'GRT'
N                   = 20
trader_distribution = 'power'
alpha               = 2
identifier          = f'{trader_distribution}_{N}'

In [ ]:
b2.get_file(f'test_data/{ticker}_impact_profile(post)_{identifier}.csv')
GRT_decay = pd.read_csv(f'{ticker}_impact_profile(post)_{identifier}.csv')
GRT_decay.head()

In [ ]:
bins = np.linspace(1, 2, 50)
z_bins_GRT = pd.cut(GRT_decay['z'], bins = bins)

dynamic_impact_bins_vals_GRT = GRT_decay['scaled impact'].groupby(z_bins_GRT, observed = False).mean()
z_bins_centers_GRT           = [interval.mid for interval in dynamic_impact_bins_vals_GRT.index.categories]
z_bins_centers_GRT           = np.array(z_bins_centers_GRT)

In [ ]:
def decay_model(z, beta, I):
    return  I * (z ** (1 - beta) - (z - 1) ** (1 - beta))

z = np.array(z_bins_centers_GRT)
impact = np.array(dynamic_impact_bins_vals_GRT)

# remove first point if z=1 exactly (can cause instability)
mask = z > 1
z_fit = z[mask]
impact_fit = impact[mask]

params, covariance = curve_fit(decay_model, z_fit, impact_fit, p0 = [0.2, 0.0005])

beta_est_GRT = params[0]
I_est_GRT    = params[1]

print('Estimated beta:', beta_est_GRT)
print('Estimated I:', I_est_GRT)


fitted_curve_GRT = decay_model(z_bins_centers_GRT, beta_est_GRT, I_est_GRT)

n   = len(z_fit)
p   = len(params)          # number of parameters (2 here)
dof = n - p                # degrees of freedom

alpha = 0.05               # 95% CI
tval  = t.ppf(1 - alpha / 2, dof)
se    = np.sqrt(np.diag(covariance))
interval_half_widths = tval * se[0], tval * se[1]
print('Interval half widths:', interval_half_widths)

I_ci        = I_est_GRT - tval * se[1], I_est_GRT + tval * se[1]
exponent_ci = beta_est_GRT - tval * se[0], beta_est_GRT + tval * se[0]
print('Scaling I 95% CI:', I_ci)
print('Exponent 95% CI:', exponent_ci)

In [ ]:
fig, ax = plt.subplots(figsize = (6, 6))

ax.scatter(z_bins_centers_GRT, dynamic_impact_bins_vals_GRT, color = 'olive', marker = 'o', label = 'GRT dynamic impact')
ax.plot(z_bins_centers_GRT, fitted_curve_GRT, color = 'red', linestyle = '--',
        label = rf'$y = Y\times\left(z^{{1-{beta_est_GRT:.2f}}} - \left(z-1\right)^{{1-{beta_est_GRT:.2f}}}\right)$')
ax.set_xlabel(r'z', fontsize = 14)
ax.set_ylabel(r'Dynamic impact $\frac{I(Q, z)}{\sigma_{D} \sqrt{Q}}$', fontsize = 14)
ax.legend()

ax.set_box_aspect(1)
plt.tight_layout()

plt.savefig(f'{ticker}_convex_profile_GRT_{identifier}.pdf')
b2.put_file(f'{ticker}_convex_profile_GRT_{identifier}.pdf', 'figures')
b2.get_file(f'figures/{ticker}_convex_profile_GRT_{identifier}.pdf')

plt.show()

In [ ]:
decay_convexity_table = pd.DataFrame({'': ['GRT'], 'scaling factor': [I_est_GRT],
                                          r'$\beta$': [beta_est_GRT]})
print(decay_convexity_table.to_latex(index = False, caption = 'Fitted values for metaorder post execution impact decay for GRT',
                                         label = 'tab:decay fitted values'))

## GFI

The impact decay of metaorders from GFI is plotted below along with the fitted curve. The point estimates of the fitted values are tabulated

In [ ]:
ticker              = 'GFI'
N                   = 20
trader_distribution = 'power'
alpha               = 2
identifier          = f'{trader_distribution}_{N}'

In [ ]:
b2.get_file(f'test_data/{ticker}_impact_profile(post)_{identifier}.csv')
GFI_decay = pd.read_csv(f'{ticker}_impact_profile(post)_{identifier}}.csv')

In [ ]:
bins = np.linspace(1, 2, 50)
z_bins_GFI = pd.cut(GFI_decay['z'], bins = bins)

dynamic_impact_bins_vals_GFI = GFI_decay['scaled impact'].groupby(z_bins_GFI, observed = False).mean()
z_bins_centers_GFI           = [interval.mid for interval in dynamic_impact_bins_vals_GFI.index.categories]
z_bins_centers_GFI           = np.array(z_bins_centers_GFI)

In [ ]:
def decay_model(z, beta, I):
    return  I * (z ** (1 - beta) - (z - 1) ** (1 - beta))

z = np.array(z_bins_centers_GFI)
impact = np.array(dynamic_impact_bins_vals_GFI)

# remove first point if z=1 exactly (can cause instability)
mask = z > 1
z_fit = z[mask]
impact_fit = impact[mask]

params, covariance = curve_fit(decay_model, z_fit, impact_fit, p0 = [0.2, 0.0005])

beta_est_GFI = params[0]
I_est_GFI    = params[1]

print('Estimated beta:', beta_est_GFI)
print('Estimated I:', I_est_GFI)


fitted_curve_GFI = decay_model(z_bins_centers_GFI, beta_est_GFI, I_est_GFI)

n   = len(z_fit)
p   = len(params)           # number of parameters (2 here)
dof = n - p                 # degrees of freedom

alpha = 0.05                # 95% CI
tval  = t.ppf(1 - alpha / 2, dof)
se    = np.sqrt(np.diag(covariance))
interval_half_widths = tval * se[0], tval * se[1]
print('Interval half widths:', interval_half_widths)

I_ci        = I_est_GFI - tval * se[1], I_est_GFI + tval * se[1]
exponent_ci = beta_est_GFI - tval * se[0], beta_est_GFI + tval * se[0]
print('Scaling I 95% CI:', I_ci)
print('Exponent 95% CI:', exponent_ci)

In [ ]:
fig, ax = plt.subplots(figsize = (6, 6))

ax.scatter(z_bins_centers_GFI, dynamic_impact_bins_vals_GFI, color = 'olive', marker = 'o', label = 'GFI dynamic impact')
ax.plot(z_bins_centers_GFI, fitted_curve_GFI, color = 'red', linestyle = '--',
        label = rf'$y = Y\times\left(z^{{1-{beta_est_GFI:.2f}}} - \left(z-1\right)^{{1-{beta_est_GFI:.2f}}}\right)$')
ax.set_xlabel(r'z', fontsize = 14)
ax.set_ylabel(r'Dynamic impact $\frac{I(Q, z)}{\sigma_{D} \sqrt{Q}}$', fontsize = 14)
ax.legend()

ax.set_box_aspect(1)
plt.tight_layout()

plt.savefig(f'{ticker}_convex_profile_GFI(20ADV).pdf')
b2.put_file(f'{ticker}_convex_profile_GFI(20ADV).pdf', 'figures')
b2.get_file(f'figures/{ticker}_convex_profile_GFI(20ADV).pdf')

plt.show()

## Table of fitted values

In [ ]:
decay_convexity_table = pd.DataFrame({'': ['GFI'], 'scaling factor': [I_est_GFI],
                                          r'$\beta$': [beta_est_GFI]})
print(decay_convexity_table.to_latex(index = False, caption = 'Fitted values for metaorder post execution impact decay for GFI',
                                         label = 'tab:decay fitted values'))